# AI Database Performance Comparicon For AI Workloads: PostgreSQL/PgVector vs MongoDB Atlas Vector Search

```
Note: This resource is intended to provide performance guidance for AI workloads using vector data within databases, this resoruce is not meant to be an official or comprehensive benchmark, but a guide to help you understand the performance characteristics of the databases within specific search patterns and workloads, enabling you to make an informed decision on which database to use for your AI workload.

Because a database can has been traditionally used for a specific workload, doesn't mean that the database is the best fit for the workload.
```

This notebook doesn't provide:
- A comprehensive benchmark fo all databases
- A cost analysis for the databases and workloads

### Introduction:
Welcome to this comprehensive notebook, where we provide performance insights for MongoDB and PostgreSQL - two of the most widely used databases in AI workloads.

In this session we analyse the performance results of a variety of search mechanisms, including:
- Vector Search
- Hybrid Search

**What You'll Learn:**
- PostgreSQL with pgvector:
  - How to set up a PostgreSQL database with the pgvector extension.
  - How to run text, vector and hybrid searches on PostgreSQL.
- MongoDB Atlas Vector Search:
  - How to set up a MongoDB Atlas database with native Vector Search capabilities.
    - How to execute text, vector and hybrid searches on MongoDB Atlas.
- AI Workload Overview:
  - This notebook showcases a standard AI workload involving vector embeddings and the retrieval of semantically simila rdocuments.
  - The system leverages two different vector search solutions:
    - PostgreSQL with pgvector: A powerful extension that integrates vector search capabilities directly into PostgreSQL.
    - MongoDB Atlas Vector Search: A native vector search feature built into MongoDB, optimized for modern, document-based applications.
- AI Workload Metrics:
  - Latency: The tiem it takes to retrieve the top `n` results
  - Throughput: The number of queriws processed per second
  - P95 Latency: The 95th percentile latency of the queries

**Database Platforms:**

For this performance guidance, we utilize:
- MongoDB Atlas: A fully managed, cloud-native database designed for modern applications.
- Neon: A serverless, fully managed PostgreSQL database optimized for cloud deployments.

Whether your focus is on MongoDB or PostgreSQl, this notebook is designed to help you understand their performance characteristics and guide you in achieving optimal performance for your AI.


## Key Information

1. **System Configuration**

### MongoDB Atlas (M30 → M40) vs. Neon (4 → 8 vCPUs) Comparison

#### Important note on Resourse Allocation Disparities

When interpreting the performance results in this notebook, it's essential to consider the significan tresource allocation differences between the tested systems:

##### MongoDB Atlas (M30 → M40)
- Minimum: 2 vCPUs, 8 GB RAM (M30)
- Maximum: 4 vCPUs, 16 GB RAM (M40)

##### Neon PosgreSQL
- Minimum: 4 vCPUs, 16 GB RAM
- Maximum: 8 vCPUs, 32 GB RAM

This means Neon PosgreSQL has **twice the compute resources** at both minimum and maximum configurations compared to MongoDB Atlas. This resource disparity significantly impacts performance results interpretation in several ways:
1. **Performance per Resource nit**: If MongoDB shows comparable or better performance despite having fewer resources, this suggests higher efficiency per compute unit.
2. **Cost Considerations**: Higher resource allocation typically incures higher costs.
3. **Scalling Behavior**: Both systems can scale, but across different resource ranges. Performance gains from scaling might manifest differently due to these distinct scaling ranges.

| Attribute                   | MongoDB Atlas (M30 → M40)                                                                 | Neon (Autoscaling: 4 → 8 vCPUs)                                                                 |
|----------------------------|---------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------|
| **vCPUs**                  | - **Min**: M30 → 2 vCPUs (8 GB RAM)  <br> - **Max**: M40 → 4 vCPUs (16 GB RAM)              | - **Min**: 4 vCPUs (16 GB RAM) <br> - **Max**: 8 vCPUs (32 GB RAM)                              |
| **Memory (RAM)**           | - **M30**: 8 GB <br> - **M40**: 16 GB                                                      | - **Min**: 16 GB <br> - **Max**: 32 GB                                                          |
| **Storage**                | - **M30**: ~40 GB included <br> - **M40**: ~80 GB included <br> (Can scale further)         | - Remote "pageserver" stores primary data <br> - Local disk for temp files: 20 GB or 15 GB × 8 CUs (whichever is higher) |
| **Autoscaling (Compute)**  | - **Cluster Tier Scaling**: can move between M30 and M40 <br> - **Storage Scaling**: automatically grows storage | - **Compute Autoscaling**: 4 to 8 vCPUs <br> - **Scale to Zero**: optional (after 5 min idle) |
| **IOPS**                   | ~2000+ on M30, higher on M40                                                               | "Burstable" IOPS from cloud storage <br> Local File Cache for frequently accessed data         |
| **Max Connections**        | - ~6000 (M30) <br> - ~12000 (M40)                                                          | - ~1802 (4 vCPUs) <br> - ~3604 (8 vCPUs)                                                       |
| **Scale to Zero**          | Not supported                                                                              | Optional. If enabled, compute suspends when idle (adds startup latency)                        |
| **Restart Behavior on Resizing** | - Moving from M30 to M40 triggers a brief re-provisioning <br> - Minimal downtime but connections can be interrupted | - Autoscaling within 4–8 vCPUs **does not** restart connections <br> - Editing min/max or toggling Scale to Zero triggers a restart |
| **Local Disk for Temp Files** | Adequate for normal ops; M40 has more local disk                                        | At least 20 GB local disk, or 15 GB × 8 CUs = 120 GB if that's higher                          |
| **Release Updates**        | - Minor updates auto-applied <br> - Major version upgrades can be scheduled               | - Weekly or scheduled updates <br> - Manual restart may be needed if Scale to Zero is disabled and you want the latest compute engine updates |


### Key Points
- **Resource Range**
  - MongoDB Atlas scales from 2 vCPUs/8 GB (M30) to 4 vCPUs/16 GB (M40)
  - Neon ranges from 4 vCPUs/16 GB to 8 vCPUs/32 GB
- **Closer Parity at M40**
  - When Atlas scales to M40, it matches Neon's minimum (4 vCPUs/16 GB), allowing more direct performance comparisons
  - Neon can still go beyond M40, up to 8 vCPUs/32 GB, if workload spikes exceed M40 capacity
- **IOPS and Connections**
  - Atlas M30→M40 has hogher IOPS and connection limits at each tier
  - Neon's IOPS is cloud-based and "burstable", while connections scale with the CPU (CUs).

In summary, MongoDB Atlas (M30 → M40) is closer to Neon (4 → 8 vCPUs) than previous tiers, especially at the high end (4 vCPUs/16 GB). However, Neon still offers more headroom if your workload demends exceed M40's capacity.

2. **Data Processing**
    - Uses Wikipedia dataset (100,000 entries) with embeddings (Precision: float32, Dimensions: 768) generated by Cohere
    - JSON data is generated from the dataset and stored in the databases
    - Stores data in both PostgreSQL and MongoDB
3. **Performance Testing**
    - Tests different sizes of concurrent queries (1-400 queries)
    - Tests different insertion batch sizes and speed of insertion

| Operation  | Metric        | Description                                                       |
|------------|---------------|-------------------------------------------------------------------|
| Insertion  | Latency       | Time taken to insert the data (average response time)            |
|            | Throughput    | Number of queries processed per second                           |
| Retrieval  | Latency       | Time taken to retrieve the top n results (average response time) |
|            | Throughput    | Number of queries processed per second                           |
|            | P95 Latency   | Time taken to retrieve the top n results for 95% of the queries  |


4. **Results Visualisation**
    - Interactive animations showing request-response cycles
    - Comparative charts for latency and throughput
    - Performance analysis across different batch sizes

## Part 1: Data Setup

Setting up the performance results dictionary `performance_guidance_results` and the batch sizes to test `CONCURRENT_QUERIES` and `TOTAL_QUERIES`
- `performance_guidance_results` is a dictionary that will store the results of the tests
- `CONCURRENT_QUERIES` is a list of the number of queries that are run concurrently
- `TOTAL_QUERIES` is the total number of queries that are run

Performance guidance Configuration Example: When testing with a concurrency level of 10:
- We run 100 iterations
- Each iteration runs 10concurrent queris
- Total queries = 1,000 queries (TOTAL_ITERATIONS*CONCURRENT_QUERIES)

NOTE: For each concurrency level in CONCURRENT_QUERIES:
1. Run TOTAL_QUERIES iterations
2. In each iteration, execute that many cuncurent queries
3. Measure and collect latencies for all queries


In [ ]:
# Initialize the performance_guidance_results results dictionary
performance_guidance_results = {"PostgreSQL": {}, "MongoDB": {}}

# The concurrency levels for performance guiddance testing
# Each level represents the number of simulteneous queries to execute
# This is important to note for AI workloads as it will affect the performance of the system as the number of requests increase
CONCURRENT_QUERIES = [
    1,
    2,
    4,
    5,
    8,
] # 24, 32, 40, 48, 50, 56, 64, 72, 80, 88, 96, 100, 200, 400

# The total number of iterations to run for each concurrency level
# Thi sis important to note for AI workloads as it will affect the performance of the system as the number of queries per request increases
TOTAL_QUERIES = 100

In [ ]:
import getpass
import os

# Function to securely get and set environment variables
def set_env_securely(var_name, prompt):
    value = getpass.getpass(prompt)
    os.environ[var_name] = value

### Step 1: Install Libraries

All the libraries are installed using pip and facilitate the sourcing of data, embedding generation, and data visualization.
- `datasets`: Hugging Face library for managing and preprocessing datasets across text, image and audio (https://huggingface.co/datasets)
- `sentence_transformers`: For creating sentence embeddings for tasks like semantic search and clustering (https://www.sbert.net/)
- `pandas`: A library for data manipulation and analysis with DataFrames and Series (https://pandas.pydata.org/)
- `matplotlib`: A library for creating static, interactive and animated data visualizations (https://matplotlib.org/)
- `seaborn`: A library for creating statistical data visualizations (https://seaborn.pydata.org/)
- `cohere`: A library for generating embeddings and accessing the Cohere API or models (https://cohere.ai/)

In [ ]:
%pip install --upgrade datasets sentence_transformers pandas matplotlib seaborn cohere

### Step 2: Data Loading

The dataset for the notebook is sourced from the Hugging Face Cohere Wikipedia dataset.

The [Cohere/wikipedia-22-12-en-embeddings](https://huggingface.co/datasets/Cohere/wikipedia-22-12-en-embeddings) dataset on Hugging Face comprises English Wikipedia articles embedded using Cohere's multilingual-22-12 model. Each entry includes the article's title, text, URL, Wikipedia ID, view count, paragraph ID, language codes, and a 768-dimensional embedding vector. This dataset is valuable for tasks like semantic search, information retrieval and NLP model training.

For this notebook we are using 100,000 rows of the dataset and have removed the id, wiki_id, paragraph_id, langs and view columns.

In [ ]:
import pandas as pd
from datasets import load_dataset

# Using 100,000 rows for testing feel free to change this to any number of rows you want to test
# The wikipedia-22-12-en-embeddings dataset has approximately 35,000,000 rows and requires 120GB of memory to load
MAX_ROWS = 100000

dataset = load_dataset(
    "Cohere/wikipedia-22-12-en-embeddings", split="train", streaming=True
)
dataset_segment = dataset.take(MAX_ROWS)

# Convert the dataset to a pandas dataframe
dataset_df = pd.DataFrame(dataset_segment)

In [ ]:
# Add a JSON attribute to the dataset consisting of the title, text and url
dataset_df["json_data"] = dataset_df.apply(
    lambda row: {"title": row["title"], "text": row["text"], "url": row["url"]}, axis=1
)

In [ ]:
# Remove the id field, wiki_id, paragraph_id, langs and views from the dataset
# This is to replicate the structure of dataset usually encountered in AI workloads, particularly particularly in RAG systems where metadata is extracted from documents and stored.
dataset_df = dataset_df.drop(
    columns=["id", "wiki_id", "paragraph_id", "langs", "views"]
)

In [ ]:
# Change the emb column name to embedding
dataset_df = dataset_df.rename(columns={"emb": "embedding"})

In [ ]:
dataset_df.head(5)

### Step 3: Embedding Generation

We use the Cohere API to generate embeddings for the test queries.

To get the Cohere API key, you can sign up for a free account on the Cohere website.

In [ ]:
# Set Cohere API key
set_env_securely("COHERE_API_KEY", "Enter your Cohere API key: ")

Using the Cohere API to generate embeddings for the test queries.

Using the `embed_multilingual-v2.0` model. This is the same model used in the Cohere Wikipedia dataset.

Embedding size is 768 dimensions and the precision is float32.

In [ ]:
from typing import List, Tuple
import cohere

# Initialize Cohere Client
co = cohere.Client()


def get_cohere_embeddings(
        sentences: List[str],
        model: str = "embed-multilingual-v2.0",
        input_type: str = "search_document",
)-> Tuple[List[float], List[int]]:
    """
    Generate embeddings for the provided sentences using Cohere's embedding model.

    Args:
    sentences (list of str): List of sentences to generate embeddings for.

    Returns:
    Tuple[List[float], List[int]]: A tuple containing two lists of embeddings (float and int8).
    """
    generated_embedding = co.embed(
        texts=sentences,
        model="embed-multilingual-v2.0",
        input_type="search_document",
        embedding_types=["float"],
    ).embeddings

    return generated_embedding.float[0]

Generate embeddings for the query templates

Store the embeddings in a dictionary for easy access

Note: Doing this to avoid tje overhead of generating embeddings for each query in the dataset during the performance analysis process, as this is a time consuming process and expensive in terms of API usage.

Note: Feel free to add more queries to the query_templates list to test the performance of the vector database with a larger number of queries

In [ ]:
query_templates = [
    "When was YouTube officially launched, and by whom?",
    "What is YouTube's slogan introduced after Google's acquisition?",
    "How many hours of videos are collectively watched on YouTube daily?",
    "Which was the first video uploaded to YouTube, and when was it uploaded?",
    "What was the acquisition cost of YouTube by Google, and when was the deal finalized?",
    "What was the first YouTube video to reach one million views, and when did it happen?",
    "What are the three separate branches of the United States government?",
    "Which counrty has the highest documented incarceration rate and prison population?",
    "How many executions have occured in the United States since 1977, and which countries have more?",
    "What percentage of the global military spending did the United States account for in 2019?",
    "How is the U.S. president elected?",
    "What cooling system innovation was included in the proposed venues for the World Cup in Qatar?",
    "What lawsuit was giled against Google in June 2022, and what was it about?",
    "How much was Google fined by CNIL in January 2022, and for what reason?",
    "When did YouTube join the NSA's PRISM program, according to reports?",    
]

# For each query template question generate the embedding
# NOTE: Doing this to avoid the overhead of generating embeddings for each query during the performance comparison
query_embeddings = [
    get_cohere_embeddings(sentences=[query], input_type="search_query")
    for query in query_templates
]

In [ ]:
# Create a dictionary with the query templates and their corresponding embeddings
query_embeddings_dict = {
    query: embedding for query, embedding in zip(query_templates, query_embeddings)
}

In [ ]:
# View the first 5 query embeddings as a dataframe
pd.DataFrame(query_embeddings_dict)

## Part 2: Retrieval Mechanisms with PostgreSQL and PgVector

In this section we create a PostgreSQL database with the PgVector extention and insert the dataset into the database.

We are also going to implement various search mechanisms on the databases to test the performance of the database under certain search patterns and workloads. Specifically we are going to implement a semantic search mechanism on the database via vector sesarch and hybrid search mechanism on the database via vector search and text search.

The table `wikipedia_data` is created with the following columns:
- `id`: The unique identifier for each row
- `title`: The title of the Wikipedia article 
- `text`: The text of the Wikipedia article
- `url`: The URL of the Wikipedia article
- `json_data`: The JSON data of the Wikipedia article
- `embedding`: The embedding vector for the Wikipedia article

The table is created with HNSW index with m=16, ef_construction=64 and cosine similarity (these are the default parameters for the HNSW index in pgvector).
- `HNSW`: Hierarchical Navigable Small World graphs are a type of graph-based index that are used for efficient similarity search.
- `m=16`: The number of edges per node in the graph
- `ef_construction=64`: Short for exploration factor construction, is the number of edges to build during the index construction phase
- `ef_search=100`: Short for exploration factor search, is the number of edges to search during the index search phase
- `cosine similarity`: The similarity metric used for the index (formula: dot product(A,B)/(|A||B|))
- `cosine distance`: The distance metric calculated using cosine similarity (1 - cosine similarity)

We perform a semantic search on the database using a single data point of the query templates and their corresponding embeddings.

### Step 1: Install Libraries

- `pgvector` (0.3.6): A PostgreSQL extension for vector similarity search (https://github.com/pgvector/pgvector)
- `psycopg` (3.2.3): A PostgreSQL database adapter for Python (https://www.psycopg.org/)

In [ ]:
%pip install --upgrade pgvector "psycopg[binary]"

### Step 2: Create Posgres Table

- `id`: The unique identifier for each row
- `title`: The title of the Wikipedia article 
- `text`: The text of the Wikipedia article
- `url`: The URL of the Wikipedia article
- `json_data`: The JSON data of the Wikipedia article
- `embedding`: The embedding vector for the Wikipedia article

**Key aspect of PostgreSQL table creation:**
- `id`: The unique identifier for each row stored with the data type `bigserial` which is a 64-bit integer and auto-incremented.
- `title`: The title of the Wikipedia article stored with the data type `text` which is a variable character string.
- `text`: The text of the Wikipedia article stored with the data type `text` which is a variable character string.
- `url`: The URL of the Wikipedia article stored with the data type `text` which is a variable character string.
- `json_data`: The JSON data of the Wikipedia article stored with the data type `jsonb` which is a binary formatted JSON data type.
- `embedding`: The embedding vector for the Wikipedia article stored with the data type `vector(768)` which is a provided by pgvector and is of 768 dimentions.

**Optimizing the table for search:**
- `search vector`: The search vector for the Wikipedia article stored with the data type `tsvector` which is a text search data type in PostgreSQL.
- The expression inside the `GENERATED ALWAYS AS` clause is the text (title and text) to be tokenized and indexed for full-text search.
- Using `coalesce` to handle any null values in the title or text columns.
- `STORED`: The keyword indicates that the `search_vector` column is stored in the table, this avoids the overhead of recalculating the `search_vector` column during queries, and improves performance.

**Extra**
- The `serach_vector` column is computed automatically using the text in the `title` and `text` fields, making full-text search more efficient by avoiding on-the-fly computation.
- The `HNSW` index on the `embedding` column is optimized for ANN queries using cosine similarity, which is crucial for semantic search.
- The `GIN` indexes on both the `json_data` and `search_vector` columns ensure fast query performance on JSONB queries and full-text search, respectively.

In [ ]:
def create_table(connection):
    with connection.cursor() as cur:
        # Drop table if it already exists
        cur.execute("DROP TABLE IF EXISTS wikipedia_data")

        # Create the table with the appropriate structure
        cur.execute("""
            CREATE TABLE wikipedia_data (
                id bigserial PRIMARY kEY,
                title text,
                text text,
                url text,
                json_data jsonb,
                embedding vector(768),
                search_vector tsvector GENERATED ALWAYS AS (
                    to_tsvector('english', coalesce(title, '') || ' ' || coalesce(text, ''))
                ) STORED
            )
        """)
        # Create HNSW index for vectorsimilarity search with cosine similarity
        cur.execute("""
            CREATE INDEX wikipedia_data_embedding_hnsw_idx
            ON wikipedia_data
            USING hnsw (embedding_vector_cosine_ops)
            WITH (m = 16, ef_construction = 64);
        """)

        # Create GIN index on the json_data column for efficient JSONB queruing
        cur.execute("""
            CREATE INDEX wikipedia_data_embedding_gin_idx
            ON wikipedia_data
            USING GIN (json_data);
        """)

        # Create GIN index on the search_vector column for efficient full-text search
        cur.execute("""
            CREATE INDEX wikipedia_data_search_vector_idx
            ON wikipedia_data
            USING GIN (search_vector);
        """)

        print("Table and indexes created successfully")
        connection.commit()

### Step 4: Define insert function

For inserting JSON data, we convert the Python Dictionary in the `json_data` attribute to a JSON string using the `json.dumps()` function. 

This is a serialization process that converts the Python Dictionary in the `json_data` attribute to a JSON string that is stored as binary data in the database.

In [ ]:
import json
import time
import numpy as np


def insert_data_to_postgres(dataframe, connection, database_type="PostgreSQL"):
    """
    Insert data into the PostgreSQL database.

    Args:
    dataframe (pandas.DataFrame): The dataframe containing the data to insert.
    connection (psycopg.extensions.connection): The connection to the PostgreSQL database.
    database_type (str): The type of database (default: "PostgreSQL"). 
    """
    start_time = time.time()
    total_rows = len(dataframe)

    try:
        with connection.cursor() as cur:
            # Create a list of tuples for insertion, filtering out rows with invalid embeddings
            data_tuples = []
            for _, row in dataframe.iterrows():
                data_tuple = (
                    row["title"],
                    row["text"],
                    row["url"],
                    json.dumps(row["json_data"]), # convert dict to JSON string
                    row["embedding"],
                )
                data_tuples.append(data_tuple)

            if not data_tuples:
                raise ValueError("No valid data tuples to insert")
            
            cur.executemary(
                """
                INSERT INTO wikipedia_data
                (title, text, url, json_data, embedding)
                VALUES (%s, %s, %s, %s, %s)
                """,
                data_tuples,
            )

            connection.commit()

    except Exception as e:
        print(f"Erroe during bulk insert: {e}")
        connection.rollback()
        raise e
    
    end_time = time.time()
    total_time = end_time - start_time
    rows_per_second = len(data_tuples) / total_time

    # print(f"\nInsertion Statistics:")
    # print(f"Total time: {total_time:.2f} seconds")
    # print(f"Average insertion rate: {rows_per_second:.2f} rows/second")
    # print(f"Total rows inserted: {len(data_tuples)}")
    # print(f"Rows skipped: {total_rows - len(data_tuples)}")

    # Store results in performance guidance dictionary
    if database_type not in performance_guidance_results:
        performance_guidance_results[database_type] = {}

    performance_guidance_results[database_type]["insert_time"] = {
        "total_time": total_time,
        "rows_per_second": rows_per_second,
        "total_rows": total_rows,
    }

In [ ]:
neon_db_host = os.environ["PGHOST"]
neon_db_database = os.environ["PGDATABASE"]
neon_db_user = os.environ["PGUSER"]
neon_db_password = os.environ["PGPASSWORD"]

### Step 5: Insert data into Postgres

In [ ]:
import psycopg
from pgvector.psycopg import register_vector

try:
    # Connect to PostgreSQL
    conn = psycopg.connect(
        f"dbname={neon_db_database} user={neon_db_user} password={neon_db_password} host={neon_db_host}"
    )

    # Enable the pgvector extension
    conn.execute("CREATE EXTENSION IF NOT EXISTS vector")

    # Register vector type to handle embedding data as vector data types
    register_vector(conn)

    # Step 1: Create the table
    create_table(conn)

    # Step 2: Insert the expended dataset into the table
    insert_data_to_postgres(dataset_df, conn)

except Exception as e:
    print("Failed to execute: ", e)
finally:
    # Close the connection
    conn.close()
    print("Connection closed")

### Step 6: Define Text search function with Postgres

In [ ]:
def text_search_with_postgres(query, connection, top_n=5):
    """
    Perform a full-text search on the precomputed 'search_vector' column of the 'wikipedia_data' table. 
    """
    with connection.cursor() as cur:
        # Convert the search query into a tsquery
        cur.execute("SELECT plainto_tsquery('english', %s)", (query,))
        ts_query = cur.fetchone()[0]

        # Execute the full-text search query using the precomputed search_vector column
        cur.execute(
            """ 
            SELECT title, text, url, json_data,
                ts_rank_cd(search_vector, %s) AS rank
            FROM wikipedia_data
            WHERE search_vector @@ %s
            ORDER BY rank DESC
            LIMIT %s
            """,
            (ts_query, ts_query, top_n),
        )

        results = cur.fetchall()

        formatted_results = [
            {
                "title": r[0],
                "text": r[1],
                "url": r[2],
                "json_data": r[3],
                "rank": r[4],
            }
            for r in results
        ]

        return formatted_results

In [ ]:
try:
    # Connect to PostgreSQL
    conn = psycopg.connect(
        f"dbname={neon_db_database} user={neon_db_user} password={neon_db_password} host={neon_db_host}"
    )

    text_search_with_postgres_results = text_search_with_postgres(
        "When was Youtube officially launched, and by whom?", conn
    )
    # Print results in a formatted way
    for result in text_search_with_postgres_results:
        print(f"\nTitle: {result}")
        print(f"Text: {result['text']}")
        print(f"URL: {result['url']}")
        print(f"JSON Data: {result['json_data']}")
        print(f"Rank: {result['rank']:.4f}")
        print("-" * 80)

except Exception as e:
    print("Failed to connect or execute query:", e)
finally:
    conn.close()
    print("Connection closed")

### Step 7: Define vector search function with Postgres

To avoid exhasuting API key usage, we will fetch the query embedding from the `query_embeddings_dict` dictionary.

In the `vector_search_with_postgres` function, we set the HNSW ef parameter to 100 using the `execute_command` function.

This is to set the exploration factor for the HNSW index to 100. And corresponds to the number of nodes/candidates to search during the index search phase. A node corresponds to a vector in the index.

In [ ]:
def vector_search_with_postgres(
        query, connection, top_n=5, filter_key=None, filter_value=None
):
    # Retrieve the query embedding from the query_embeddings_dict
    query_embedding = query_embeddings_dict[query]

    with connection.cursor() as cur:
        # Set the HNSW ef parameter
        cur.execute("SET hnsw.ef_search = 100")
        connection.commit()

        # Construct the base SQL query
        sql_query = """ 
            SELECT title, text, url, json_data,
                embedding <=> %s::vector AS similarity
            FROM wikipedia_data
        """

        # Append the filter condition if provided
        if filter_key and filter_value:
            sql_query += " WHERE json_data->>%s = %s"

        # Append the ORDER BY and LIMIT clauses
        sql_query += """ 
            ORDER BY similarity ASC
            LIMIT %s
        """

        # Prepare the parameters for the query
        params = [query_embedding]
        if filter_key and filter_value:
            params.extend([filter_key, filter_value])
        params.append(top_n)

        # Execute the query with parameters
        cur.execute(sql_query, params)

        # Fetch and return the top results
        results = cur.fetchall()

        # Format results as a list of dictionaries for easier handling
        formatted_results = [
            {
                "title": r[0],
                "text": r[1],
                "url": r[2],
                "json_data": r[3],
                "similarity": r[4],
            }
            for r in results
        ]

        return formatted_results

In [ ]:
# Connect to PostgreSQL
try:
    conn = psycopg.connect(
        f"dbname={neon_db_database} user={neon_db_user} password={neon_db_password} host={neon_db_host}"
    )

    # Run semantic search with a sample query
    query_text = "When YouTube officially launched, and by whom?"
    results = vector_search_with_postgres(
        query_text, conn, top_n=5, filter_key="title", filter_value="YouTube"
    )

    # Print results in a formatted way
    for result in results:
        print(f"\nTitle: {result['title']}")
        print(f"Text: {result['text']}")
        print(f"URL: {result['url']}")
        print(f"JSON Data: {result['json_data']}")
        print(f"Similarity Score: {1- result['similarity']:.4f}")
        print("-" * 80)

except Exception as e:
    print("Failed to connect or execute query:", e)
finally:
    conn.close()
    print("Connection closed")

### Step 8: Define hybrid search function with Postgres

In [ ]:
from psycopg import sql


def hybrid_search_with_postgres(
        query, connection, top_n=5, filter_key=None, filter_value=None
):
    """ 
    Perform a hybrid search combining semantic vector similarity and full-text search.

    Args:
        query (str): The search query string.
        connection: A psycopg2 database connction object.
        top_n (int): Number of top results to return (default is 5).
        filter_key (str, optional): JSON key to filter rusults on.
        filter_value (str, optional): Value of the JSON key to filter results.

    Returns:
        list: A list of dictionaries containing the search results.
    """
    # Retrieve the query embedding from the query_embeddings_dict
    query_embedding = query_embeddings_dict[query]

    with connection.cursor() as cur:
        # Set the HNSW ef_search parameter for vecotr search 
        cur.execute("SET ivfflat.probes = 10")
        connection.commit()

        # Base SQL components
        base_vector_query = sql.SQL("""
            SELECT id, title, text, url, json_data,
                    embedding <=> %s::vector AS vector_similarity
            FROM wikipedia_data
        """)
        base_full_text_query = sql.SQL("""
            SELECT id, title, text, url, json_data,
                    ts_rank_cd(search_vector, plainto_tsquery('english', %s)) AS text_rank
            FROM wikipedia_data
            WHERE search_vector @@ plainto_tsquery('english', %s)
        """)

        # Initialize parameters list
        vector_params = [query_embedding]
        text_params = [query, query]

        # Append filter condition if provided
        if filter_key and filter_value:
            filter_condition = sql.SQL("json_data->>{} = %s").format(
                sql.Literal(filter_key)
            )
            base_vector_query += sql.SQL(" WHERE ") + filter_condition
            base_full_text_query += sql.SQL(" AND ") + filter_condition
            vector_params.append(filter_value)
            text_params.append(filter_value)

        # Execute the vector similarity search 
        cur.execute(base_vector_query + sql.SQL(" LIMIT %s"), vector_params + [top_n])
        vector_results = cur.fetchall()

        # Execute the full-text search
        cur.execute(base_full_text_query + sql.SQL(" LIMIT %s"), text_params + [top_n])
        text_results = cur.fetchall()

        # Combine and rank results using Reciprocal Rank Fusion (RRF)
        combined_results = {}
        rrf_k = 60 # RRF parameter; adjust as needed

        # Process vector results
        for rank, row in enumerate(vector_results, start=1):
            doc_id = row[0]
            if doc_id not in combined_results:
                combined_results[doc_id] = {
                    "title": row[1],
                    "text": row[2],
                    "url": row[3],
                    "json_data": row[4],
                    "vector_similarity": row[5],
                    "text_rank": 0,
                    "rrf_score": 0,
                }
            combined_results[doc_id]["rrf_score"] += 1 / (rrf_k + rank)

        # Process full_text results
        for rank, row in enumerate(text_results, start=1):
            doc_id = row[0]
            if doc_id not in combined_results:
                combined_results[doc_id] = {
                    "title": row[1],
                    "text": row[2],
                    "url": row[3],
                    "json_data": row[4],
                    "vector_similarity": None,
                    "text_rank": row[5],
                    "rrf_score": 0,
                }
            combined_results[doc_id]["rrf_score"] += 1 / (rrf_k + rank)

        # Sort combined results by RRF score in descending order
        sorted_results = sorted(
            combined_results.values(), key=lambda x: x["rrf_score"], reverse=True
        )

        # Return the top_n results
        return sorted_results[:top_n]

In [ ]:
try:
    # Connect to PostgreSQL
    conn = psycopg.connect(
        f"dbname={neon_db_database} user={neon_db_user} password={neon_db_password} host={neon_db_host}"
    )

    # Run hybrid search with a sample query
    query_text = "When was YouTube officially launched, and by whom?"
    results = hybrid_search_with_postgres(
        query_text, conn, top_n=5, filter_key="title", filter_value="YouTube"
    )
    for result in results:
        print(f"\nTitle: {result['title']}")
        print(f"Text: {result['text']}")
        print(f"URL: {result['url']}")
        print(f"JSON Data: {result['json_data']}")
        if result["vector_similarity"] is not None:
            print(f"Vector Similarity Score: {1 - result['vector_similarity']:.4f}")
        if result["text_rank"] > 0:
            print(f"Text Rank: {result['text_rank']:.4f}")
        print("-" * 80)

except Exception as e:
    print("Failed to connect or execute query: ", e)
finally:
    conn.close()
    print("Connection closed")

## Part 3: Retrieval Mechanisms with MongoDB Atlas

### Step 1: Install Libraries

- `pymongo` (4.10.1): A pyton driver for MongoDB (https://pymongo.readthedocs.io/en/stable/)

In [ ]:
%pip install --quiet --upgrade pymongo

### Step 2: Create MongoDB Atlas Account

TODO: Place infoiant required

### Step 3: Connect to MongoDB and Create Database and Collection

In [ ]:
# Set MongoDB URI
# Example: mongodb+srv://<db_username>:<db_password>@cluster0.wi4s1.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0
set_env_securely("MONGO_URI", "Enter your MONGO URI: ")

In the following code blocks below we do the following:

1. Establish a connection to the MongoDB database
2. Create a database and connection if they do not already exist
3. Delete all data in the collection if it already exists

In [ ]:
import pymongo

def get_mongo_client(mongo_uri):
    """Establish and validate connection to the MongoDB."""

    client = pymongo.MongoClient(
        mongo_uri, appname="devrel.showcase.postgres_neon_vs_mongodb_atlas.python"
    )

    # Validate the connection
    ping_result = client.admin.command("ping")
    if ping_result.get("ok") == 1.0:
        # Connection successful
        print("Connection to MongoDB successful")
        return client
    else:
        print("Connection to MongoDB failed")
    return None


MONGO_URI = os.environ["MONGO_URI"]
if not MONGO_URI:
    print("MONGO_URI not set in environment variables")

In [ ]:
from pymongo.errors import CollectionInvalid

mongo_client = get_mongo_client(MONGO_URI)

DB_NAME = "vector_db"
COLLECTION_NAME = "wikipedia_data"

# Create or get the database
db = mongo_client[DB_NAME]

# Check if the collection exists
if COLLECTION_NAME not in db.list_collection_names():
    try: # Create the collection
        db.create_collection(COLLECTION_NAME)
        print(f"Collection '{COLLECTION_NAME}' created successfully.")
    except CollectionInvalid as e:
        print(f"Error creating collection: {e}")
else:
    print(f"Collection '{COLLECTION_NAME}' already exists.")

# Assign the collection
collection = db[COLLECTION_NAME]

In [ ]:
collection.delete_many({})

### Step 4: Vector Index Creation

The `setup_vecto_search` function creates a vector search index for the MongoDB collection.

The `index_name` parameter is the name of the index to create.

The `embedding_field_name` parameter is the name of the field containing the text embeddings on each document within the wikipedia_data collection.

In [ ]:
embedding_field_name = "embedding"
vector_search_index_name = "vector_index"

Filtering your data is useful to narrow the scope of your semantic search and ensure that not all vectors are considered for comparison. It reduces the number of documents against which to run similarity comparisons, which can decrease query latency and increase the accuracy of search results.

You must index the fields that you want to filter by using the filter type inside the fields array.

In [ ]:
import time
from pymongo.operations import SearchIndexModel

def setup_vector_search_index(collection, index_name="vector_index"):
    """
    Setup a vector search index for MongoDB collection.

    Args:
    collection: MongoDB collection object
    index_definition: Dictionary containing the index definition
    index_name: Name of the index (default: "vector_index") 
    """
    new_vector_search_index_model = SearchIndexModel(
        definition={
            "fields": [
                {
                    "type": "vector",
                    "path": "embedding",
                    "numDimentions": 768,
                    "similarity": "cosine",
                },
                {
                    "type": "filter",
                    "path": "json_data_title",
                },
            ]
        },
        name=index_name,
        type="vectorSearch",
    )

    # Create the new index
    try:
        result = collection.create_search_index(model=new_vector_search_index_model)
        print(f"Creating index '{index_name}'...")

        return result
    
    except Exception as e:
        print(f"Error creating new vector search index '{index_name}': {e!s}")
        return None

In [ ]:
setup_vector_search_index(collection, "vector_index")

An Atlas Search index is a data structure that categorizes data in an easily searchable format. It is a mapping between terms and the documents that contain those terms. Atlas Search indexes enable faster retrieval of documents using certain identifiers. You must configure an Atlas Search index to query data in your Atlas cluster using Atlas Search.

You can create an Atlas Search index on a single field or on multiple fields. We recommend that you index the fields that you regularly use to sort or filter your data in order to quickly retrieve the documents that contain the retrieval data at query-time.

In [ ]:
def setup_text_search_index(collection, index_name="tetx_search_index"):
    """
    Setup a text search index for a MongoDB collection in Atlas.

    Args:
        uri (str): MongoDB Atlas connection string.
        database_name (str): Name of the database.
        collection_name (str): Name of the collection.
        index_name (str): Name of the index (default: "text_search_index")
    """
    # Define the search index model
    search_index_model = SearchIndexModel(
        definition={
            "mappings": {
                "dynamic": True # Index all fields dynamically
            },
        },
        name=index_name,
        type="search",
    )

    # Create the search index
    try:
        result = collection.create_search_index(model=search_index_model)
        print(f"Creating index '{index_name}'...")

        # Wait for the index to be created
        time.sleep(30)
        print(f"30-second wait completed for index '{index_name}'.")

        print(f"Index '{index_name}' created successfully.")
        return result
    
    except Exception as e:
        print(f"Error creating text search index '{index_name}': {e}")
        return None

In [ ]:
setup_text_search_index(collection, "text_search_index")

### Step 5: Define Insert Data Function

Because of the affinity of MongoDB for JSON data, we don't have to convert the Python Dictionary in the `json_data` attribute to a JSON string using the `json.dumps()` function. Instead, we can directly insert the Python Dictionary into the MongoDB collection.

This reduced the operational overhead of the insertion processes in AI workloads.

In [ ]:
def insert_data_to_mongodb(dataframe, collection, database_type="MongoDB"):
    start_time = time.time()
    total_rows = len(dataframe)

    try:
        # Convert DataFrame to list of dictionaries for MongoDB insertion
        documents = dataframe.to_dict("records")

        # Use insert_many for better performance
        result = collection.insert_many(documents)

        end_time = time.time()
        total_time = end_time - start_time
        rows_per_second = total_rows / total_time

        # print(f"\nMongoDB Insertion Statistics:")
        # print(f"Total time: {total_time:.2f} seconds")
        # print(f"Average insertion rate: {rows_per_second:.2f} rows/second")
        # print(f"Total rows inserted: {len(result.inserted_ids)}")

        # Store results in performance guidance dictionary
        if database_type not in performance_guidance_results:
            performance_guidance_results[database_type] = {}

        performance_guidance_results[database_type]["insert_time"] = {
            "total_time": total_time,
            "rows_per_second": rows_per_second,
            "total_rows": total_rows,
        }

        return True
    
    except Exception as e:
        print(f"Error during MongoDB insertion: {e}")
        return False

### Step 6: Insert Data into MongoDB

### Step 7: Define Text Search Function

### Step 8: Define Vector Search Function

### Step 9: Define Hybrid Search Function

#### Note: 
In the MongoDB hybrid search function, two weights:

## Part 4: Vector Database Performance Analysis Process

#### Insertion Performance Analysis Process

1.1 PosgreSQL Insertion Performance Analysis

1.2 MongoDB Insertion Performance Analysis

1.3 Visualize Insertion Performance Analysis

#### Semantic Search with PostgreSQL and PgVector Performance Analysis

2.2. MongoDB Semantic Search Performance Analysis

2.3 Visualize Vector Search Performance Analysis

## Part 5: Extra Notes

5.1 PostgreSQL JSONB vs. MongoDB BSON

5.2 Limitations of pgvector for Handling Large-Dimensional Embeddings